### 1. 데이터 크롤링

- 빅카인즈에서 4대 신문사의 4개의 각 카테고리에서 5개의 키워드를 사용해서 10년치 신문 기사를 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import NoSuchElementException, StaleElementReferenceException
import os
import time
import pandas as pd
import numpy as np
import re
import json

In [ ]:
class BigkindsCrawler:
    def __init__(self, path, year):
        self.df = pd.read_csv(path, encoding="utf-8-sig")
        self.year = year
        # 각 월 별 날짜 수 (Hard-coded) & 각 월
        self.months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
        self.days = ["31", "28", "31", "30", "31", "30", "31", "31", "30", "31", "30", "31"]
        self.options = webdriver.ChromeOptions()

    # set the WebDriver options
    def set_driver_options(self):
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        # set User-Agent for preventing access blocked
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)" +
                                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
        # prevent webdriver from closing immediately
        self.options.add_experimental_option("detach", True)
        # 크롬 브라우저가 직접적으로 열리지 않도록 설정
        self.options.add_argument('--headless')
        # 불필요한 이미지 로딩 없앰 (시간 단축)
        self.options.add_argument('--disable-logging')
        self.options.add_argument('--disable-images')

    # csv 파일 필요: publisher, keyword를 csv로 먹임
    # category: 통합 분류 (li: 정치=1, 경제=2, 사회=3, 국제=5)
    def executor(self, publisher, m, category, keyword):
        res = []

        start_day = self.year + "-" + self.months[m] + "-" + "01"
        end_day = self.year + "-" + self.months[m] + "-" + self.days[m]

        # webdriver 생성
        driver = webdriver.Chrome(options=self.options)
        driver.get("https://www.bigkinds.or.kr/v2/news/index.do")

        # 언론사 클릭
        pub = self.transform_publisher(publisher)
        driver.find_element(By.XPATH, f"//*[@id='category_provider_list']/li[{pub}]/span/label").click()
        time.sleep(0.5)

        # 기간 클릭 (배너)
        driver.find_element(By.XPATH, "//*[@id='collapse-step-1-body']/div[3]/div/div[1]/div[1]/a").click()
        # 기간 클릭 (1개월)
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[1]/span[3]/label").click()

        # 시작 날짜 클릭
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[2]/div/div[1]/img").click()
        start = driver.find_element(By.XPATH, "//*[@id='search-begin-date']")
        start.send_keys(Keys.CONTROL, 'a')
        start.send_keys(start_day)

        # 종료 날짜 클릭
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[2]/div/div[3]/img").click()
        end = driver.find_element(By.XPATH, "//*[@id='search-end-date']")
        end.send_keys(Keys.CONTROL, 'a')
        end.send_keys(end_day)
        time.sleep(0.5)

        # 통합 분류 클릭 (배너)
        # 그냥 클릭하면 페이지 로딩 시간 때문에 오류가 날 수 있어서 webdriver 기다림
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='collapse-step-1-body']/div[3]/div/div[2]/div[1]/a"))
        )
        element.click()

        # 통합 분류 (li: 정치=1, 경제=2, 사회=3, 국제=5)
        driver.find_element(By.XPATH, f"//*[@id='srch-tab3']/ul/li[{category}]/div/span[4]").click()
        time.sleep(1)

        # 키워드 입력: 오류 나지 않게 한 글자씩 입력함
        keyword_input = driver.find_element(By.XPATH, "//*[@id='total-search-key']")
        for k in keyword:
            keyword_input.send_keys(k)
            time.sleep(0.2)
        keyword_input.send_keys(Keys.RETURN)
        time.sleep(1)

        # 정확도순
        driver.find_element(By.XPATH, "//*[@id='select1']/option[2]").click()
        time.sleep(1)

        try:
            # 맨 위의 기사 클릭
            driver.find_element(By.XPATH, "//*[@id='news-results']/div[1]/div/div[2]").click()
            time.sleep(1)

            # "일자", "언론사", "제목", "URL", "본문"
            try:
                date = driver.find_element(By.XPATH,
                                           "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/div[1]/ul/li[1]").text
            except NoSuchElementException:
                date = "N/A"

            title = driver.find_element(By.XPATH, "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/h1").text

            # URL 오류 처리
            href_button = driver.find_element(By.XPATH,
                                              "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/div[2]/div[1]/button[1]")

            if href_button.text == "기사원문":
                href = href_button.get_attribute("onclick")

                # ?가 포함되었을 경우, 쿼리 문자열이므로 뒤의 문자열은 삭제
                try:
                    url = re.search(r'https?://[^?]+', href).group()
                except AttributeError:
                    # URL이 매치되지 않는 경우, 예외 처리를 통해 http 이후의 문자열만 저장
                    url = re.search(r'https?://+', href).group()
            else:
                url = "N/A"

            paper = driver.find_element(By.XPATH, "//*[@id='news-detail-modal']/div/div/div[1]/div/div[2]")
            main_text = paper.text
        except (StaleElementReferenceException, NoSuchElementException):
            date = "N/A"
            publisher = "N/A"
            title = "N/A"
            url = "N/A"
            main_text = "N/A"

        res.append(date)
        res.append(publisher)
        res.append(title)
        res.append(url)
        res.append(main_text)

        driver.quit()

        print(res)
        return res

    # 월 단위별로 크롤링
    def crawling(self, MONTH):
        crawled_df = pd.DataFrame(columns=["일자", "언론사", "제목", "URL", "본문"])

        s_index = (MONTH - 1) * 16
        size = 16
        publishers = self.df.loc[s_index:s_index + size, "언론사"]

        categories = self.df.loc[s_index:s_index + size, "카테고리"]

        total_time = 0

        for i in range(s_index, s_index + size):
            rank_str = self.df.loc[i, "top-10 키워드"]
            rank_str = rank_str.replace("'", '"')

            # JSON 문자열을 파이썬 리스트로 변환
            data_list = json.loads(rank_str)
            data_list = data_list[:5]

            keywords = [item['name'] for item in data_list]

            publisher = publishers[i]
            category = categories[i] // 1000000

            # 변수 체크용
            print(f"CSV 행 = {i}")
            print(f"언론사 = {publisher}")
            print(f"카테고리 = {category}")

            for j, keyword in enumerate(keywords):
                s = time.time()
                print(
                    f"{'|||||  * Process: ' + str(MONTH) + 'th month ' + str(i * 5 + j + 1) + 'th/' + '960th *  |||||':^50}")
                crawled_df.loc[i * 5 + j, :] = self.executor(publisher, MONTH - 1, category, keyword)
                e = time.time()
                total_time += round(e - s, 2)
                print(f"Elapsed time: {total_time:.2f}")
                time.sleep(1)

        return crawled_df

    def transform_publisher(self, p):
        pub = 0
        if p == "경향신문":
            pub = 1
        elif p == "동아일보":
            pub = 4
        elif p == "조선일보":
            pub = 8
        elif p == "중앙일보":
            pub = 9
        elif p == "한겨레":
            pub = 10

        return pub

In [ ]:
# 2015년 크롤링
YEAR = "2015"

# 1 ~ 12월까지 크롤링하고, 각 월 별로 데이터 프레임을 만듭니다.
for Month in range(6, 7):
    crawler = BigkindsCrawler(f"topkeywords_{YEAR}.csv", YEAR)
    crawler.set_driver_options()  # 옵션 세팅

    dataframe = crawler.crawling(Month)
    dataframe.to_csv(f"{YEAR}_{Month}.csv", encoding="utf-8-sig")

    print("Waiting..")
    time.sleep(5)
    # 메모리 삭제 후 재할당을 위한 코드
    del crawler

### 2. 데이터 전처리

In [ ]:
import os
os.chdir('C:/Users/user/PD')

In [ ]:
# 데이터를 언론사 별로 분리
def seperate_by_press(df):
    JoonAng_news = df[df['언론사'] == '중앙일보']
    DongA_news = df[df['언론사'] == '동아일보']
    Hani_news = df[df['언론사'] == '한겨레']
    Kyung_news = df[df['언론사'] == '경향신문']
    return JoonAng_news, DongA_news, Hani_news, Kyung_news

In [ ]:
# 신문 기사 텍스트 전처리
def cleaning_text(text):
    # text가 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    # (/br)로 나오는 경우 제거
    text = text.replace('(/br)', '')
    # 특수 문자 지정 후 제거
    text = re.sub(r'[☞▶◆#⊙※△▽▼□■◇◎☎○]+', '', text, flags=re.UNICODE)
    text = re.sub(r'〃', '', text)  
    # 한자 및 일본어 제거
    text = re.sub(r'[\p{Script=Hiragana}\p{Script=Katakana}\p{Script=Han}]+', '', text, flags=re.UNICODE)
    # 이메일 주소 제거
    text = re.sub(r'\S+@\S+', '', text)
    # 사이트 주소 제거(www. 으로 시작하고 .kr로 끝나는 경우)
    text = re.sub(r'www\..+\.kr', '', text)
    
    # 대괄호로 둘러싸인 내용을 삭제 (10글자 미만인 경우는 삭제, 10글자 이상인 경우는 유지)
    text = re.sub(r'\[([^\]]{1,9})\]', '', text)
    text = re.sub(r'\[([^\]]{10,})\]', r'\1', text)
    # "<...>"로 둘러싸인 내용을 삭제
    text = re.sub(r'<[^>]*>', '', text)
    # 소괄호 안에 아무런 내용도 없으면 삭제
    text = re.sub(r'\(\s*\)', '', text)

    return text

def preprocess_news(df, column_name = '본문'):
    df[column_name] = df[column_name].apply(cleaning_text)
    return df

In [ ]:
# 경로 설정
data_dir = 'C:/Users/user/PD/input/2014'  # 데이터 디렉토리 경로
output_dir = 'C:/Users/user/PD/output/2014'  # 출력 디렉토리 경로
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성 (없으면)

# 년도와 월에 대한 범위 설정
start_year = 2014
end_year = 2014
months = range(1, 13)

# 월별로 데이터 처리
for year in range(start_year, end_year + 1):
    for month in months:
        # 데이터 파일명 생성
        file_name = f'{year}_{month}.csv'
        data_path = os.path.join(data_dir, file_name)
        
        # 데이터 불러오기
        news = pd.read_csv(data_path, encoding='utf-8')
        
        # 전처리 수행
        news = preprocess_news(news)
        
        # 전처리된 데이터 저장
        output_file_name = f'processed_{file_name}'
        output_path = os.path.join(output_dir, output_file_name)
        news.to_csv(output_path, encoding='utf-8-sig', index=False)
        print(f'{file_name} 전처리 및 저장 완료')